# Handling Missing Data

In [1]:
import numpy as np
import pandas as pd

## Missing Data in Pandas

Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floatingpoint NaN value, and the Python None object. None is a Python object, it cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type 'object' (i.e., arrays of Python objects)

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
48.8 ms ± 621 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
4.18 ms ± 289 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [4]:
# vals1.sum() will give error because addition between an integer and None is undefined

The other missing data representation, NaN (acronym for Not a Number), is different. It is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation.

In [5]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [6]:
1 + np.nan

nan

In [7]:
0 * np.nan

nan

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(np.float64(nan), np.float64(nan), np.float64(nan))

NumPy does provide some special aggregations that will ignore missing values

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(np.float64(8.0), np.float64(1.0), np.float64(4.0))

NaN and None both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate.

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don’t have an available sentinel value, Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to np.nan, it will automatically be upcast to a floating-point type to accommodate the NA.

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

| Type Class | Conversion when storing NAs | NA Sentinel Value |
| --- | --- | --- |
| floating | No change | np.nan |
| object | No change | None or np.nan |
| integer | Cast to float64 | np.nan |
| boolean | Cast to object | None or np.nan |

## Operating on Null Values

In [13]:
data = pd.Series([1, np.nan, 'hello', None])

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [15]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

In [16]:
data[data.notnull()]

0        1
2    hello
dtype: object

In [17]:
data.dropna() # Removes NA values

0        1
2    hello
dtype: object

In [18]:
df = pd.DataFrame([[1, np.nan, 2],
                   [2, 3, 5],
                   [np.nan, 4, 6]]
                 )
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [19]:
df.dropna() # Drop all rows in which any null value is present

,0,1,2
1,2.0,3.0,5


In [20]:
df.dropna(axis=1) # Drop all columns in which any null value is present

,2
0,2
1,5
2,6


In [21]:
df.dropna(axis='rows') # Drop all rows in which any null value is present (equivalent to axis=0)

,0,1,2
1,2.0,3.0,5


In [22]:
df.dropna(axis='columns') # Drop all columns in which any null value is present (equivalent to axis=1)

,2
0,2
1,5
2,6


In [23]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [24]:
df.dropna(axis='columns', how='all') # Drop all columns in which all values are null

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [25]:
df.dropna(axis='rows', thresh=3) # Drop all rows which have less than 3 non-null values

,0,1,2,3
1,2.0,3.0,5,NaN


In [26]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [27]:
data.fillna(0) # Fill NA entries with a single value, 0 in this case

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [28]:
data.ffill() # forward-fill

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [29]:
data.bfill() # backward-fill

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [30]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [31]:
df.ffill(axis=1) # forward-fill along columns

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
